# Ejercicios propuestos: *Networking*

Los siguientes problemas se dejan como opción para ejercitar los conceptos revisados sobre *networking* (semana 12-13). Si tienes dudas sobre algún problema o alguna solución, no dudes en dejar una issue en el foro del curso.

## Ejercicio 0: Analizar el código del servidor y del cliente TCP

Lee con atención la estructura básica de la comunicación cliente-servidor de los contenidos, la necesitarás para los ejercicios que vienen ;)

## Ejercicio 1: Haciendo "eco" a un cliente

El malvado emperador EnZurg, hermano gemelo malvado de Enzo, ha desterrado a Enzo a las lejanas tierras del DCCampo. Hace un tiempo, por razones desconocidas, EnZurg está intentando contactar de nuevo Enzo a través de su servidor privado, pero este no quiere responderle y prefiere hacerlo perder el tiempo por haberlo desterrado. En este ejercicio deberás completar el servidor de Enzo.

El servidor de Enzo funciona de una forma bastante especial. Cada vez que el cliente de EnZurg que se conecta le envía un mensaje, Enzo responde el mismo contenido (un "eco") pero con mayúsculas y minúsculas puestas al azar, deformando el *string*. Algo así:

> EnZurg: Hola, Enzo
> 
> Enzo: HoLa, eNzO

Para esto, Enzo te entrega su función `deformador_string`. Finalmente, como Enzo creció junto con EnZurg, conoce a la perfección el código del cliente que está utilizando su hermano (podría serte útil para probar tu servidor). **Es tiempo de completar el servidor para molestar a EnZurg.**

In [ ]:
# Código funcion deformador_string
import random


def deformador_string(string):
    string_deformado = ""
    for caracter in string:
        if random.random() <= 0.5:
            string_deformado += caracter.upper()
        else:
            string_deformado += caracter.lower()
    return string_deformado

print(deformador_string("Hola, Enzo"))


In [ ]:
# Código cliente de EnZurg
import socket


# Creo un socket TCP
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# Así puedo obtener el hostname de la máquina actual
host = socket.gethostname()
port = 9001

# Me conecto al socket del servidor de Enzo
sock.connect((host, port))

try:
    while True:
        # Escribo el mensaje a enviarle a Enzo
        mensaje = input("Yo a Enzo: ")
        # Si escribo "salir", se corta la conexión
        if mensaje.lower() == "salir":
            sock.sendall("salir".encode("utf-8"))
            break
        # Envio mi mensaje al servidor
        sock.sendall(mensaje.encode("utf-8"))
        # Recibo la respuesta que me envíe
        data = sock.recv(4096)
        print("Enzo a mi:", data.decode("utf-8"))
except ConnectionError as e:
    print("Ocurrió un error.")
finally:
    sock.close()


In [ ]:
# Código servidor de Enzo
import random
import socket

sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# Así podemos obtener el hostname de la máquina actual
host = socket.gethostname()
port = 9001

# Dejamos el socket esperando ("escuchando") por conexiones
sock.bind((host, port))
sock.listen()

# Aqui aceptamos la conexión de EnZurg
socket_cliente, address = sock.accept()


def deformador_string(string):
    string_deformado = ""
    for caracter in string:
        if random.random() <= 0.5:
            string_deformado += caracter.upper()
        else:
            string_deformado += caracter.lower()
    return string_deformado


# COMPLETAR AQUI
# Debes recibir el mensaje de EnZurg,
# aplicarle la función deformador_string
# y enviar ese string modificado de vuelta a EnZurg


## Ejercicio 2: Envío de mensajes desde el servidor

En un nuevo intento por saber de Enzo, EnZurg decide dejar de intentar contactarlo y simplemente esperar a que le envíe él un mensaje. Es por esto que se te ocurre ayudar a Enzo enviando *spam* al cliente de EnZurg que se intentará conectar al servidor de Enzo, simplemente para llenarlo de basura e información no muy útil.

En este ejercicio, deberás volver a implementar un servidor, pero deberá enviar un mensaje con *spam* **cada 5 segundos** al cliente que se conecte.

In [ ]:
# Código cliente de EnZurg
import socket

# Creo un socket TCP
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# Así puedo obtener el hostname de la máquina actual
host = socket.gethostname()
port = 9002

# Me conecto al socket del servidor de Enzo
sock.connect((host, port))

try:
    while True:
        # Voy a ir recibiendo sus mensajes
        data = sock.recv(4096)
        # Y ahora los voy a ir imprimiendo
        print("Enzo a mi:", data.decode("utf-8"))
except ConnectionError as e:
    print("Ocurrió un error.")
finally:
    sock.close()


In [ ]:
# Código servidor de Enzo
import random
import socket
import time

sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# Así podemos obtener el hostname de la máquina actual
host = socket.gethostname()
port = 9002

# Dejamos el socket esperando ("escuchando") por conexiones
sock.bind((host, port))
sock.listen()

# Aqui aceptamos la conexión de EnZurg
socket_cliente, address = sock.accept()


MENSAJES_SPAM = [
    "SPAM!! SPAM!! SPAM!! SPAM!! SPAM!!",
    "lalalalalalalalalalalalala",
    "Intentando sobrecargar el cliente de EnZurg",
    "La respuesta a la Vida, el Universo, y Todo lo Demás es... 42",
]

# COMPLETAR AQUI
# Cada 5 segundos debes enviar un mensaje
# de la lista MENSAJES_SPAM a EnZurg


## Ejercicio 3: Definiendo un protocolo más formal de comunicación

Finalmente, Enzo decide aceptar conversar con EnZorg. Como su relación es bastante complicada, deciden establecer un protocolo de comunicación formal para no tener problemas de comunicación en el futuro. Lamentablemente, como ambos son bastante tercos, este sistema que podría ser bastante simple termina siendo más complejo de lo necesario (algunos dicen que por su complejidad podría ser utilizado en una tarea).

Las reglas del protocolo son las siguientes:
* Los primeros 4 bytes de un mensaje indicarán el largo del contenido del mensaje, los que tendrán que estar en formato *little endian*
* El contenido del mensaje debe ser dividido en tantos bloques de 80 *bytes* como sea necesario, y para cada bloque se deberán anteponer 4 *bytes* que indiquen el número de bloque, codificado en *big endian*.
* Para transformar *strings* a *bytes* se deberá utilizar UTF-8.

En este ejercicio, deberás primero hacer funciones que permitan trabajar fácilmente con el protocolo (parte A) y luego implementar un cliente y un servidor que utilice dicho protocolo (parte B), que serán utilizados por Enzo y EnZurg desde ahora en adelante.

### Parte A: Implementación del protocolo

Primero que todo, el [*endiannes*](https://es.wikipedia.org/wiki/Endianness) se refiere al orden en que se guardan los *bytes* que representan un número. Dado que un número puede requerir representarse con más de un *byte*, es importante saber si primero tenemos los bytes más significativos o los menos significativos. 

Cuando convertimos un número a *bytes*, podemos elegir cuántos bytes utilizar (primer argumento) y el *endiannes* (`byteorder`) de su representación (segundo argumento):

In [4]:
(1025).to_bytes(4, byteorder="little") # little endian

b'\x01\x04\x00\x00'

In [5]:
(1025).to_bytes(4, byteorder="big") # big endian

b'\x00\x00\x04\x01'

Como puedes ver, estamos representando el número 1025 con 4 *bytes* en ambas celdas, pero el orden de los *bytes* en cada celda cambia.

A continuación, se hará un ejemplo paso a paso del protocolo.

In [1]:
# Este es el mensaje de ejemplo (sí, es una receta para hacer sopaipillas)
MENSAJE = ("Lo primero que hay que tener en cuenta para saber cómo hacer "
           "sopaipillas, es hacer un “volcán” con la harina dejando un hueco "
           "al medio, vierta al centro la manteca derretida con la leche o el "
           "agua, la sal y el zapallo previamente molido hasta formar una "
           "pasta suave, mezcle todo hasta formar una masa suave y elástica, "
           "esta no debe tener grumos y no se tiene que pegar a la mesa.\n"
           "\n"
           "El siguiente paso para saber cómo hacer sopaipillas es amasar la "
           "masa con un uslero o a mano dejándola de aproximadamente 5mm de "
           "grosor, luego  cortarla en  círculos de 10cm (esto es a gusto, "
           "pueden ser más gruesas y más pequeñas), se recomienda pincharlas "
           "con un tenedor para que al momento de freírlas la masa no se "
           "arruine.\n"
           "\n"
           "El último paso es freír la masa en un sartén profundo y caliente "
           "el aceite a fuego alto, para probar el aceite lance un pequeño "
           "trozo de masa, debe burbujear y flotar en la superficie, coloque "
           "de 2 a 3 sopaipillas y fríalas 1 minuto por lado, no se deben "
           "dorar demasiado. Una vez fritas se sacan de la freidora y dejan en "
           "papel absorbente.\n"
           "\n"
           "Luego de haber leído esta receta de comida típica chilena, usted "
           "ya habrá sabido cómo hacer sopaipillas.")

**Paso 0:** Convertir el contenido a *bytes* (usando UTF-8)

**Paso 1:** Obtener el largo del contenido y representar dicho largo en 4 *bytes* (*little endian*). Este es el **primer bloque**

**Paso 2:** Dividir el contenido en bloques de 80 *bytes*, y mantener el orden de los bloques
* **Paso 2a:** En este mismo paso, se puede convertir el índice del bloque a su representación en el protocolo (4 *bytes* en formato *big endian*)
* **Paso 2b:** Conservar cada bloque (ya sea de contenido o su posición) en su posición correcta (a partir del primer bloque)

> *Hint*: Recuerda la sección chunks del primer notebook de la semana pasada

Ahora, deberás crear **una función `codificar`** que retorne la lista de bloques a enviar, con la codificación del contenido ya explicada.

In [2]:
def codificar(contenido):
    contenido_como_bytes = None # Paso 0 (COMPLETAR)
    largo = len(contenido_como_bytes)
    largo_como_bytes = None # Paso 1 (COMPLETAR)
    BLOQUES = [largo_como_bytes]
    for ... in ...: # Paso 2 (COMPLETAR)
        posicion_como_bytes = None # Paso 2a (COMPLETAR)
        chunk_como_bytes = None # Paso 2 (COMPLETAR)
        BLOQUES.append(posicion_como_bytes) # Paso 2b
        BLOQUES.append(chunk_como_bytes) # Paso 2b
    return BLOQUES

In [3]:
codificar(MENSAJE)

[b'\x99\x04\x00\x00',
 b'\x00\x00\x00\x00',
 b'Lo primero que hay que tener en cuenta para saber c\xc3\xb3mo hacer sopaipillas, es ha',
 b'\x00\x00\x00\x01',
 b'cer un \xe2\x80\x9cvolc\xc3\xa1n\xe2\x80\x9d con la harina dejando un hueco al medio, vierta al centro l',
 b'\x00\x00\x00\x02',
 b'a manteca derretida con la leche o el agua, la sal y el zapallo previamente moli',
 b'\x00\x00\x00\x03',
 b'do hasta formar una pasta suave, mezcle todo hasta formar una masa suave y el\xc3\xa1s',
 b'\x00\x00\x00\x04',
 b'tica, esta no debe tener grumos y no se tiene que pegar a la mesa.\n\nEl siguiente',
 b'\x00\x00\x00\x05',
 b' paso para saber c\xc3\xb3mo hacer sopaipillas es amasar la masa con un uslero o a man',
 b'\x00\x00\x00\x06',
 b'o dej\xc3\xa1ndola de aproximadamente 5mm de grosor, luego  cortarla en  c\xc3\xadrculos de ',
 b'\x00\x00\x00\x07',
 b'10cm (esto es a gusto, pueden ser m\xc3\xa1s gruesas y m\xc3\xa1s peque\xc3\xb1as), se recomienda ',
 b'\x00\x00\x00\x08',
 b'pincharlas con 

### Parte B: Usando el protocolo en el servidor y el cliente

Ahora, basandote en el código de cliente y servidor utilizado en los pasos anteriores, deberás implementar un servidor y un cliente que utilicen el protocolo. Presta especial atención al uso de los datos en cada parte. Como notarás, deberás recibir primero los bytes que indiquen el largo del mensaje, estimar la cantidad de bloques y para cada bloque recibir su respeciva posición y contenido.